# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingenieria de Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Proyecto Final**: Structured Streaming kafka

**Fecha**:13/05/2025

**Nombre del Estudiante**:David Abraham Naranjo Salgado, Benjamin Leonardo Zarate Solano y Angel David Cortes Pacheco

**Profesor**: Pablo Camarillo Ramirez

In [24]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [25]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://873bad4e62fe:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

### Creación del Kafka Stream

In [26]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "fc2fc5ae9bbc:9093") \
                .option("subscribe", "kafka-spark-producer-A, kafka-spark-producer-B , kafka-spark-producer-C") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [27]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))


### Configuración del "Sink" del stream

In [28]:
!pwd

/home/jovyan


In [30]:
query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("parquet") \
                .option("truncate", "false") \
                .option("path","/home/jovyan/data") \
                .option("checkpointLocation", "/home/jovyan/checkpoint_data") \
                .start()

query.awaitTermination(300)

# Revisa estado del stream después
if query.exception() is not None:
    print("⚠️ Error:", query.exception())


25/05/11 01:24:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/11 01:24:10 WARN StreamingQueryManager: Stopping existing streaming query [id=c0d9692d-a438-459d-aa20-50a4bd537270, runId=18531a9e-650c-4355-a702-f5466f0e26d9], as a new run is being started.
25/05/11 01:24:10 ERROR FileFormatWriter: Aborting job 85c8dbac-1d42-4e70-a5d0-c87cf776a474.
java.lang.InterruptedException
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1048)
	at scala.concurrent.impl.Promise$DefaultPromise.tryAwait0(Promise.scala:243)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:255)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:104)
	at org.apache.spark.util.ThreadUtils$.awaitReady(ThreadUtils.scala:342)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:980)
	at org.apache.spark.SparkC

KeyboardInterrupt: 

In [32]:
query.stop()
sc.stop()